## Titanic dataset processing

```
TD:

Kategoristen muuttujien slice
Age uudelleenkoodaus
Cabin -> splittaa niin saa kannen
Ticketissä ylimääräistä skeidaa
Tsekkaa tuplarivit (koko aineisto & vain matkustaja-avaimet)
Nimet, ticketit pois, rivi-indekseiksi matkustaja-id:t
Nullien käsittely, jos yksikään halutuissa sarakkeissa, pois (agessa paljon) -> mallit ilman ikää ja ilman nulleja?
Boxplotit (esim. embarked & age)
```

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression

In [6]:
# Whole train dataset

df = pd.read_csv('train.csv', sep=";")

display(df.iloc[:10,:])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


###Descriptive visualizations

In [7]:
# Some formatting, duplicate and null checks

print(f"Shape: {df.shape}")
print(f"Numerical columns:  {df.median().index.values}, categorical columns: {df.index.values[df.median().index.values]}")

# Check for PassengerId duplicates:

print(f"PassengerId duplicates: {len(df['PassengerId'])-len(df['PassengerId'].drop_duplicates())}")

# Nulls per column:

print(f"Nulls:  {df.isnull().sum()}")

# Decode column Sex to bool:

df["Sex_Bool"] = df["Sex"].replace({'male': 0, 'female': 1})
df = df[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Sex_Bool', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]

# General passenger data visualization:

# val = df.value_counts("Sex")
# plt.bar(x=list(val.index.values), height=val.to_list())
# plt.plot()

# Tähän subplot histogrammeja ikä, sukupuoli, luokka, kansi?

# Seuraavaan voisi laittaa samoja mutta erotella selviytynyt-statuksen perusteella

# Malli:

# fig, axs = plt.subplots(2,2, figsize=(15,10))

# axs[0,0].hist(miehet.sort_values()[:195], bins=25)
# axs[0,0].set_title('Histogrammi / miehet')

# axs[0,1].hist(naiset.sort_values()[:195], bins=25, color='r')
# axs[0,1].set_title('Histogrammi / naiset')

# sm.qqplot(miehet, line='q', ax=axs[1,0])
# axs[1,0].set_title('QQ-plot / miehet')

# sm.qqplot(naiset, line='q', markerfacecolor='r', markeredgecolor='r', ax=axs[1,1])
# axs[1,1].set_title('QQ-plot / naiset')

# plt.show()

Shape: (891, 12)


TypeError: Cannot convert [['Braund, Mr. Owen Harris'
  'Cumings, Mrs. John Bradley (Florence Briggs Thayer)'
  'Heikkinen, Miss. Laina' ... 'Johnston, Miss. Catherine Helen "Carrie"'
  'Behr, Mr. Karl Howell' 'Dooley, Mr. Patrick']
 ['male' 'female' 'female' ... 'female' 'male' 'male']
 ['A/5 21171' 'PC 17599' 'STON/O2. 3101282' ... 'W./C. 6607' '111369'
  '370376']
 [nan 'C85' nan ... nan 'C148' nan]
 ['S' 'C' 'S' ... 'S' 'C' 'Q']] to numeric